In [16]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from time import sleep

In [18]:
from tqdm.notebook import tqdm, tnrange, tqdm_notebook

import matplotlib.pyplot as plt

import numpy as np

import torch.nn as nn

from torchvision import datasets,transforms

In [19]:
from fastai.imports import *
from fastai.torch_core import *

In [20]:
path = Path('/storage')
path = path/'mnist'

In [21]:
path.mkdir(exist_ok=True)

In [22]:
train = datasets.MNIST(path,
                       download=True,
                       transform=transforms.Compose([transforms.ToTensor()]))
test = datasets.MNIST(path,
                      train = False ,
                      download=True,
                      transform=transforms.Compose([transforms.ToTensor()]))

In [23]:
trainset = torch.utils.data.DataLoader(train, batch_size=32,shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=32,shuffle=True)

In [24]:
def RMSELoss(yhat,y):
    return torch.sqrt(torch.nn.functional.mse_loss(yhat,y))
#     return torch.sqrt(torch.mean((yhat-y)**2))

criterion = RMSELoss


In [25]:
def vae_loss(orignal,reconstructed):
    pred = reconstructed.get('model_output')
    
    data_fidelity_loss = orignal * torch.log(1e-10 + pred) + (1 - orignal) * torch.log(1e-10 + 1 - pred)
    data_fidelity_loss = -torch.sum(data_fidelity_loss)
    
    mu, sigma = reconstructed.get('mu'), reconstructed.get('sigma')
    
    kl_loss = 1  + sigma - torch.pow(mu,2) - torch.exp(sigma)
    kl_loss = -0.5 * torch.sum(kl_loss)
    
    a,b = 1,1
    
    return (a*data_fidelity_loss + b*kl_loss) /2
    
criterion = vae_loss

In [26]:
class net(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.latent_size = 11
        self.l1 = nn.Linear(28*28,499)
        self.l2 = nn.Linear(499,359)
        self.l3 = nn.Linear(359,209)
        self.l4 = nn.Linear(209,99)
#         self.l5 = nn.Linear(99,self.latent_size)
        self.mu = nn.Linear(99,self.latent_size)
        self.sigma = nn.Linear(99,self.latent_size)
        self.l6 = nn.Linear(self.latent_size,99)
        self.l7 = nn.Linear(99,209)
        self.l8 = nn.Linear(209,359)
        self.l9 = nn.Linear(359,499)
        self.l10 = nn.Linear(499,28*28)
        
        self.fulyCnctdLayer = [self.l1, self.l2,
                               self.l3, self.l4, 
                               self.mu, self.sigma, 
                               self.l6, self.l7, 
                               self.l8, self.l9,
                              self.l10]
        
        self.drpt_layers = [nn.Dropout(.5),nn.Dropout(.5),nn.Dropout(.5),nn.Dropout(.5),nn.Dropout(.5),nn.Dropout(.5),
                     nn.Dropout(.5),nn.Dropout(.5),nn.Dropout(.5),nn.Dropout(.5)]
        
        self.bthNrm_layers = [nn.BatchNorm1d(499), nn.BatchNorm1d(359), nn.BatchNorm1d(209), nn.BatchNorm1d(99),
                                nn.BatchNorm1d(self.latent_size), nn.BatchNorm1d(self.latent_size),  
                                 nn.BatchNorm1d(99), nn.BatchNorm1d(209), nn.BatchNorm1d(359), nn.BatchNorm1d(499),]
    
        
    def basic_layer(self,x,layer_n,sigmoid=False):
        L_layer  = self.fulyCnctdLayer[layer_n]
        L_bthNrm = self.bthNrm_layers[layer_n]
        L_drpt = self.drpt_layers[layer_n]
        x = (L_bthNrm(L_drpt(L_layer(x))))
        if not sigmoid: return torch.relu(x)
        else: return torch.sigmoid(x)
               
    def forward(self,x):
        x = self.basic_layer(x,0)
        x = self.basic_layer(x,1)
        x = self.basic_layer(x,2)
        x = self.basic_layer(x,3)
        mu = self.basic_layer(x,4)
        sigma = self.basic_layer(x,5)
        epsilon = torch.empty(self.latent_size).normal_(mean=0.,std=1.)
        latent = mu + torch.exp(.5 + sigma)*epsilon
        x = self.basic_layer(latent,6)
        x = self.basic_layer(x,7)
        x = self.basic_layer(x,8)
        x = self.basic_layer(x,9)
        x = torch.sigmoid(self.l10(x))
        return {'model_output' : x ,
               'mu' : mu,
               'sigma' : sigma}
        
#     def forward(self,x):
#         x = self.l1(x).relu()
#         x = self.l2(x).relu()
#         x = self.l3(x).relu()
#         x = self.l4(x).relu()
# #         x = self.l5(x).relu()

#         mu_layer = self.mu(x).relu()
#         sigma = self.sigma(x).relu()
#         epsilon = torch.empty(11).normal_(mean=0.,std=1.)
#         latent = mu_layer + torch.exp(.5 + sigma)*epsilon
#         latent = latent

#         x = self.l6(latent).relu()
#         x = self.l7(x).relu()
#         x = self.l8(x).relu()
#         x = self.l9(x).relu()
#         x = torch.sigmoid(self.l10(x))
#         return x

    
    
        
        

In [30]:
model = net()
# print(model)

In [ ]:
opt = torch.optim.Adam(model.parameters(),lr = 3e-3, weight_decay = 3e-3 )

EPOCHS  = 100

#scheduler = torch.optim.lr_scheduler.OneCycleLR(opt, max_lr=9e-3, steps_per_epoch=len(trainset), epochs=EPOCHS)

scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(opt, 10, 1, eta_min = 1e-4)

for e in tnrange(EPOCHS, desc='EPOCHS', leave='False', unit='Epoch'):
    model.train()

    for data in tqdm(trainset, desc='Training Loop', leave='False', unit='batch'):
        x,y = data
        model.zero_grad()
        out = model(x.view(-1,28*28))
        loss = criterion(x.view(-1,28*28),out)
        loss.backward()
        opt.step()
    scheduler.step()
    
    loss_batch = [] 
    
    for data in tqdm(testset, desc='Test Loop', leave='False', unit='batch'):
        model.eval()
        x,y = data
        with torch.no_grad():
            out = model(x.view(-1,28*28))
            loss = criterion(x.view(-1,28*28),out)
            loss_batch.append(loss.detach().numpy())
        
    tqdm_notebook.write(f'Epoch {e+1} loss => {np.mean([loss_batch])}')


Epoch 1 loss => 3163.627685546875



Epoch 2 loss => 3131.873291015625



Epoch 3 loss => 3137.18603515625



Epoch 4 loss => 3107.596923828125



Epoch 5 loss => 3096.37890625


...


Epoch 49 loss => 2954.40087890625



Epoch 50 loss => 2946.508056640625



Epoch 51 loss => 3030.493896484375



Epoch 52 loss => 3020.577392578125



Epoch 53 loss => 3025.276123046875


opt = torch.optim.Adam(model.parameters(),lr = 1e-3, weight_decay = 3e-3 )

EPOCHS  = 100

for e in tnrange(EPOCHS, desc='EPOCHS', leave='False', unit='Epoch'):
    model.train()
    for data in tqdm(trainset, desc='Training Loop', leave='False', unit='batch'):
        x,y = data
        model.zero_grad()
        out = model(x.view(-1,28*28))
        loss = criterion(x.view(-1,28*28),out)
        loss.backward()
        opt.step()
        
    loss_batch = [] 
    
    for data in tqdm(testset, desc='Test Loop', leave='False', unit='batch'):
        model.eval()
        x,y = data
        with torch.no_grad():
            out = model(x.view(-1,28*28))
            loss = criterion(x.view(-1,28*28),out)
            loss_batch.append(loss.detach().numpy())
        
    tqdm_notebook.write(f'Epoch {e+1} loss => {np.mean([loss_batch])}')

In [14]:
torch.save(model.state_dict(),path/'model_100_epoch')
torch.save(opt.state_dict(),path/'opt_100_epoch')


torch.save(model.state_dict(),path/'model_700_epoch')
torch.save(opt.state_dict(),path/'opt_700_epoch')


model.load_state_dict(torch.load(path/'model_20_epoch'))
opt.load_state_dict(torch.load(path/'opt_20_epoch'))


In [ ]:
opt = torch.optim.Adam(model.parameters(),lr = 1e-3)

EPOCHS  = 700

for e in tnrange(EPOCHS, desc='EPOCHS', leave='False', unit='Epoch'):
    model.train()
    for data in tqdm(trainset, desc='Training Loop', leave='False', unit='batch'):
        x,y = data
        model.zero_grad()
        out = model(x.view(-1,28*28))
        loss = criterion(x.view(-1,28*28),out)
        loss.backward()
        opt.step()
        
    loss_batch = [] 
    
    for data in tqdm(testset, desc='Test Loop', leave='False', unit='batch'):
        model.eval()
        x,y = data
        with torch.no_grad():
            out = model(x.view(-1,28*28))
            loss = criterion(x.view(-1,28*28),out)
            loss_batch.append(loss.detach().numpy())
        
    tqdm_notebook.write(f'Epoch {e+1} loss => {np.mean([loss_batch])}')


Epoch 1 loss => 3079.813720703125



Epoch 2 loss => 3065.605712890625



Epoch 3 loss => 3064.958984375



Epoch 4 loss => 3057.221923828125



Epoch 5 loss => 3052.587158203125


...


Epoch 48 loss => 3028.398193359375



Epoch 49 loss => 3016.725830078125



Epoch 50 loss => 3029.08544921875



Epoch 51 loss => 3020.8876953125



Epoch 52 loss => 3028.5107421875


torch.save(model.state_dict(),path/'model_1000_epoch')
torch.save(opt.state_dict(),path/'opt_1000_epoch')


opt = torch.optim.Adam(model.parameters(),lr = 1e-3,weight_decay=1e-2)

EPOCHS  = 1000

for e in range(EPOCHS):
    for data in trainset:
        x,y = data
        model.zero_grad()
        out = model(x.view(-1,28*28))
        loss = criterion(x.view(-1,28*28),out)
        loss.backward()
        opt.step()
        
    loss_batch = [] 
    
    for data in testset:
        x,y = data
        model.zero_grad()
        out = model(x.view(-1,28*28))
        loss = criterion(x.view(-1,28*28),out)
        loss_batch.append(loss.detach().numpy())
        
    print(np.mean([loss_batch]))

In [ ]:
out = 0
for data in testset:
    x,y = data
    out = model(x.view(-1,28*28))
    break


plt.imshow(np.reshape(out.get('model_output')[0].detach().numpy(),(-1,28)))

model.load_state_dict(torch.load(path/'model_20_epochs'))

opt = torch.optim.Adam(model.parameters(),lr = 3e-3)

opt.load_state_dict(torch.load(path/'optim_20_epochs'))


EPOCHS  = 1

for e in range(EPOCHS):
    for data in trainset:
        x,y = data
        model.zero_grad()
        out = model(x.view(-1,28*28))
        loss = criterion(x.view(-1,28*28),out)
        loss.backward()
        opt.step()
        
    loss_batch = [] 
    
    for data in testset:
        x,y = data
        out = model(x.view(-1,28*28))
        loss = criterion(x.view(-1,28*28),out)
        loss_batch.append(loss.detach().numpy())
        
    print(np.mean([loss_batch]))


EPOCHS  = 1

for e in range(EPOCHS):
    for data in trainset:
        x,y = data
        model.zero_grad()
        out = model(x.view(-1,28*28))
        loss = criterion(x.view(-1,28*28),out)
        loss.backward()
        opt.step()
        
    loss_batch = [] 
    
    for data in testset:
        x,y = data
        out = model(x.view(-1,28*28))
        loss = criterion(x.view(-1,28*28),out)
        loss_batch.append(loss.detach().numpy())
        
    print(np.mean([loss_batch]))


EPOCHS  = 5

for e in range(EPOCHS):
    for data in trainset:
        x,y = data
        model.zero_grad()
        out = model(x.view(-1,28*28))
        loss = criterion(x.view(-1,28*28),out)
        loss.backward()
        opt.step()
        
    loss_batch = [] 
    
    for data in testset:
        x,y = data
        out = model(x.view(-1,28*28))
        loss = criterion(x.view(-1,28*28),out)
        loss_batch.append(loss.detach().numpy())
        
    print(np.mean([loss_batch]))